In [1]:
import audiocraft

A matching Triton is not available, some optimizations will not be enabled.
Error caught was: No module named 'triton'
C:\Users\hrayr\anaconda3\envs\MusicGen\lib\site-packages\deeplake\util\check_latest_version.py:32: UserWarning: A newer version of deeplake (3.9.11) is available. It's recommended that you update to the latest version using `pip install -U deeplake`.
  warnings.warn(


In [2]:
from audiocraft.models import MusicGen

In [5]:
musicgen = MusicGen.get_pretrained('facebook/musicgen-small')

C:\Users\hrayr\anaconda3\envs\MusicGen\lib\site-packages\torch\nn\utils\weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")
